In [1]:
import numpy as np
import pandas as pd
import psycopg2 as pg2

#### Load data

In [2]:
df = pd.read_csv('./data/aq_state_data_2019.csv')

In [4]:
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)

In [5]:
df.head()

,aqi,arithmetic_mean,cbsa,cbsa_code,city,county,county_code,date_local,date_of_last_change,datum,event_type,first_max_hour,first_max_value,latitude,local_site_name,longitude,method,method_code,observation_count,observation_percent,parameter,parameter_code,poc,pollutant_standard,sample_duration,site_address,site_number,state,state_code,units_of_measure,validity_indicator
0,50.0,0.044417,"Birmingham-Hoover, AL",13820.0,Birmingham,Jefferson,73,2019-08-31,2020-03-10,WGS84,None,9,0.054,33.553056,North Birmingham,-86.815,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,87.0,12,71.0,Ozone,44201,1,Ozone 8-hour 2015,8-HR RUN AVG BEGIN HOUR,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",23,Alabama,1,Parts per million,N
1,50.0,0.039211,"Birmingham-Hoover, AL",13820.0,Birmingham,Jefferson,73,2019-08-31,2020-03-10,WGS84,None,9,0.054,33.553056,North Birmingham,-86.815,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,87.0,19,79.0,Ozone,44201,1,Ozone 8-Hour 2008,8-HR RUN AVG BEGIN HOUR,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",23,Alabama,1,Parts per million,Y
2,50.0,0.039211,"Birmingham-Hoover, AL",13820.0,Birmingham,Jefferson,73,2019-08-31,2020-03-10,WGS84,None,9,0.054,33.553056,North Birmingham,-86.815,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,87.0,19,79.0,Ozone,44201,1,Ozone 8-Hour 1997,8-HR RUN AVG BEGIN HOUR,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",23,Alabama,1,Parts per million,Y
3,NaN,0.035333,"Birmingham-Hoover, AL",13820.0,Birmingham,Jefferson,73,2019-08-31,2020-03-10,WGS84,None,10,0.059,33.553056,North Birmingham,-86.815,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,87.0,24,100.0,Ozone,44201,1,Ozone 1-hour 1979,1 HOUR,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",23,Alabama,1,Parts per million,Y
4,90.0,0.040706,"Birmingham-Hoover, AL",13820.0,Birmingham,Jefferson,73,2019-08-30,2020-03-10,WGS84,None,10,0.067,33.553056,North Birmingham,-86.815,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,87.0,17,100.0,Ozone,44201,1,Ozone 8-hour 2015,8-HR RUN AVG BEGIN HOUR,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",23,Alabama,1,Parts per million,Y


#### Leave data that only matters: AQI of Ozone and PM2.5 at every measuring point of all states

In [9]:
df = df.dropna(axis=0)[df['validity_indicator'] == 'Y']\
    [['aqi', 'date_local', 'latitude', 'longitude', 'parameter', 'state']]

C:\Users\jjing\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [10]:
df.shape

(2714610, 6)

In [11]:
df.head()

,aqi,date_local,latitude,longitude,parameter,state
1,50.0,2019-08-31,33.553056,-86.815,Ozone,Alabama
2,50.0,2019-08-31,33.553056,-86.815,Ozone,Alabama
4,90.0,2019-08-30,33.553056,-86.815,Ozone,Alabama
5,90.0,2019-08-30,33.553056,-86.815,Ozone,Alabama
6,90.0,2019-08-30,33.553056,-86.815,Ozone,Alabama


In [14]:
df['parameter'].unique()

array(['Ozone', 'PM10 Total 0-10um STP', 'PM2.5 - Local Conditions',
       'Carbon monoxide', 'Nitrogen dioxide (NO2)', 'Sulfur dioxide'],
      dtype=object)

In [16]:
df = df[(df['parameter'] == 'Ozone') | (df['parameter'] == 'PM2.5 - Local Conditions')]

In [19]:
df['parameter'] = df['parameter'].map(lambda x: 'PM25' if x == 'PM2.5 - Local Conditions' else 'OZ')

In [22]:
df = df.pivot_table(index=['latitude', 'longitude', 'date_local', 'parameter', 'state'], values=['aqi'], aggfunc='mean')
df.reset_index(level=['latitude', 'longitude', 'date_local', 'parameter', 'state'], inplace=True)
df.head()

,latitude,longitude,date_local,parameter,state,aqi
0,19.117561,-155.778136,2019-01-01,PM25,Hawaii,24.0
1,19.117561,-155.778136,2019-01-02,PM25,Hawaii,20.0
2,19.117561,-155.778136,2019-01-03,PM25,Hawaii,21.0
3,19.117561,-155.778136,2019-01-04,PM25,Hawaii,25.0
4,19.117561,-155.778136,2019-01-05,PM25,Hawaii,24.0


#### Prepare data to create tables in a database

In [28]:
df_point = df.pivot_table(index=['latitude', 'longitude'], values=['aqi'], aggfunc='mean')
df_point.reset_index(level=['latitude', 'longitude'], inplace=True)
df_point = df_point.drop(columns=['aqi'])
df_point['pt_id'] = [x for x in range(1482)]
df_point.tail()

,latitude,longitude,pt_id
1477,61.598898,-149.106220,1477
1478,64.762973,-147.310297,1478
1479,64.840672,-147.722461,1479
1480,64.845690,-147.727413,1480
1481,64.845930,-147.693270,1481


In [31]:
df = df.merge(df_point, on=['latitude', 'longitude'], how='left')[['date_local', 'parameter', 'state', 'aqi', 'pt_id']]
df.head()

,date_local,parameter,state,aqi,pt_id
0,2019-01-01,PM25,Hawaii,24.0,0
1,2019-01-02,PM25,Hawaii,20.0,0
2,2019-01-03,PM25,Hawaii,21.0,0
3,2019-01-04,PM25,Hawaii,25.0,0
4,2019-01-05,PM25,Hawaii,24.0,0


#### Make a database

In [108]:
db_pwd = 'password'
conn = pg2.connect(database='AQI_2019', user='postgres', password=db_pwd)
cur = conn.cursor()

In [104]:
# create table
create_args = \
"""CREATE TABLE measuring_pt (
lat float NOT NULL, 
long float NOT NULL, 
pt_id integer PRIMARY KEY
)"""

cur.execute(create_args)
conn.commit()

In [109]:
create_args = \
"""CREATE TABLE aqi (
date VARCHAR(10) NOT NULL,
type VARCHAR(16) NOT NULL,
state VARCHAR(32) NOT NULL,
aqi float NOT NULL,
pt_id integer NOT NULL
)"""

cur.execute(create_args)
conn.commit()

In [106]:
# a function insert a dataframe into a table
def save_database(cursor, table, df):
    values = []
    for row in df.itertuples(index=False):
        values.append(tuple([str(x) for x in row]))
    args_str = str(values).replace('[', '').replace(']', '')
    cursor.execute("INSERT INTO " + table + " VALUES " + args_str)
    conn.commit()

In [110]:
save_database(cur, 'aqi', df)

In [111]:
save_database(cur, 'measuring_pt', df_point)

In [115]:
# fetch data
cur.execute("""SELECT * FROM aqi
            INNER JOIN measuring_pt ON measuring_pt.pt_id = aqi.pt_id
            WHERE aqi.date='2019-09-01'
            LIMIT 50""")
pd.DataFrame(cur.fetchall(), columns=['date', 'type', 'state', 'aqi', 'id1', 'lat', 'long', 'pt_id']).drop(columns=['id1'])

,date,type,state,aqi,lat,long,pt_id
0,2019-09-01,OZ,Texas,51.0,29.043759,-95.472946,76
1,2019-09-01,OZ,Florida,12.0,29.170533,-82.100646,77
2,2019-09-01,OZ,Florida,12.0,29.192754,-82.173149,78
3,2019-09-01,OZ,Texas,80.0,29.254474,-94.861289,80
4,2019-09-01,PM25,Texas,47.0,29.254474,-94.861289,80
5,2019-09-01,OZ,Texas,44.0,29.275381,-98.311692,81
6,2019-09-01,PM25,Texas,29.0,29.275381,-98.311692,81
7,2019-09-01,OZ,Texas,48.0,29.515090,-98.620166,83
8,2019-09-01,PM25,Texas,31.0,29.515090,-98.620166,83
9,2019-09-01,OZ,Texas,64.0,29.520443,-95.392509,84


In [116]:
cur.close()
conn.close()